# 04 Transfer Entropy

In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

import jupyter_black

jupyter_black.load()

In [2]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import src.cleansing as cleansing
from loguru import logger

In [3]:
df = pd.read_csv("../data/Weather Data.csv")

# rename columns
# parse datetime column
df = cleansing.clean_weather_data(df)

In [5]:
# Transfer Entropy uses the conditional mutual information formulation
from src.conditional_mutual_information import get_conditional_mutual_information

## The lags

In [6]:
df.head()

,date_time,temp_c,dew_point_temp_c,real_hum_pct,wind_speed_kmh,visibility_km,press_kpa,weather
0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog


We observed that:
- `temp_c` and `dew_point_temp_c` have high positive correlation
- `visibility_km` and `real_hum_pct` have high negative correlation
- `dew_point_temp_c` and `date_time` also appear to have some correlation

Now, we are going to test their causality relationships.

But first, we need to define the lags - from our interpretation, what lags can we use to find the different causality relationships? 